# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{
    'input_ids': tensor([
        [  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172, 2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,     0,     0,     0,     0,     0,     0]
    ]), 
    'attention_mask': tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    ])
}

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 2])

In [ ]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [1]:
!pip install numpy pandas scikit-learn matplotlib seaborn

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # Essential for Gradient Descent
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the California Housing dataset
california_housing = fetch_california_housing(as_frame=True)
X = california_housing.data  # Features
y = california_housing.target # Target: median house value (in hundreds of thousands of dollars)

# 2. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Crucial for Gradient Descent: Feature Scaling (Standardization) ---
# Gradient Descent works best when features are on a similar scale.
# StandardScaler transforms data to have zero mean and unit variance.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # Use the same scaler fitted on training data for test data

# Convert back to DataFrame for easier handling if needed (optional for core algorithm)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [8]:
type(X_train_scaled)

numpy.ndarray

In [5]:

# 3. Implement Linear Regression with Gradient Descent from Scratch

class LinearRegressionGD:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None # Coefficients (beta_1 to beta_n)
        self.bias = None    # Intercept (beta_0)
        self.cost_history = [] # To track MSE over iterations

    def fit(self, X, y):
        # Add bias (intercept) term to X
        # We add a column of ones to X, so the first weight (beta_0) can be learned
        # directly along with other weights.
        X_b = np.c_[np.ones((X.shape[0], 1)), X] # Add column of 1s for bias term

        n_samples, n_features = X_b.shape # n_features now includes the bias term

        # Initialize weights (coefficients) to zeros or small random values
        self.weights = np.zeros(n_features) # Includes bias as the first weight
        print()

        # Reshape y to be a 2D array if it's 1D
        y = y.values.reshape(-1, 1) if isinstance(y, pd.Series) else y.reshape(-1, 1)


        # Gradient Descent loop
        for iteration in range(self.n_iterations):
            # Calculate predictions (hypothesis)
            y_predicted = X_b.dot(self.weights.reshape(-1, 1))

            # Calculate the error (residuals)
            errors = y_predicted - y

            # Calculate gradients (partial derivatives of cost function w.r.t. each weight)
            # Derivative of MSE is (2/m) * sum((y_pred - y_actual) * X_j)
            gradients = (2/n_samples) * X_b.T.dot(errors)

            # Update weights
            self.weights -= self.learning_rate * gradients.flatten() # Flatten gradients to 1D

            # Store cost (MSE) for monitoring convergence
            cost = np.mean(errors**2) # MSE
            self.cost_history.append(cost)

        # Separate bias from other weights for clarity
        self.bias = self.weights[0]
        self.weights = self.weights[1:]

    def predict(self, X):
        # Add bias term to X for prediction
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        # Use learned weights and bias to make predictions
        return X_b.dot(np.concatenate(([self.bias], self.weights))).flatten()



In [6]:
# --- Train and Evaluate the Custom Model ---
print("--- Training Custom Linear Regression (Gradient Descent) ---")
# Instantiate our custom model
model_gd = LinearRegressionGD(learning_rate=0.01, n_iterations=2000) # Increased iterations

# Fit the model to the scaled training data
model_gd.fit(X_train_scaled, y_train) # Pass numpy arrays, not DataFrames for internal ops

print(f"\nCustom Model - Intercept (beta_0): {model_gd.bias:.4f}")
print("Custom Model - Coefficients (beta_i for each feature):")
for feature, coef in zip(X.columns, model_gd.weights):
    print(f"  {feature}: {coef:.4f}")


# Make predictions on the scaled test data
y_pred_gd = model_gd.predict(X_test_scaled)


# Evaluate the custom model's performance
mse_gd = mean_squared_error(y_test, y_pred_gd)
rmse_gd = np.sqrt(mse_gd)
r2_gd = r2_score(y_test, y_pred_gd)

print("\n--- Custom Model Evaluation on Test Set ---")
print(f"Mean Squared Error (MSE): {mse_gd:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_gd:.4f}")
print(f"R-squared (R2 Score): {r2_gd:.4f}")



--- Training Custom Linear Regression (Gradient Descent) ---

Custom Model - Intercept (beta_0): 2.0719
Custom Model - Coefficients (beta_i for each feature):
  MedInc: 0.8648
  HouseAge: 0.1298
  AveRooms: -0.3039
  AveBedrms: 0.3423
  Population: 0.0001
  AveOccup: -0.0418
  Latitude: -0.8332
  Longitude: -0.8069

--- Custom Model Evaluation on Test Set ---
Mean Squared Error (MSE): 0.5558
Root Mean Squared Error (RMSE): 0.7455
R-squared (R2 Score): 0.5759


In [ ]:
# --- Visualize Cost History ---
plt.figure(figsize=(10, 6))
plt.plot(range(len(model_gd.cost_history)), model_gd.cost_history, color='blue')
plt.title("Cost History (MSE) during Gradient Descent Training")
plt.xlabel("Number of Iterations")
plt.ylabel("Mean Squared Error (MSE)")
plt.grid(True)
plt.show()

# --- Visualize Predictions vs. Actual ---
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_gd, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2) # Perfect prediction line
plt.xlabel("Actual Median House Value ($100,000s)")
plt.ylabel("Predicted Median House Value ($100,000s)")
plt.title("Actual vs. Predicted House Values (Custom Linear Regression GD)")
plt.grid(True)
plt.show()



In [ ]:
# --- Compare with Scikit-learn's Linear Regression (for sanity check) ---
from sklearn.linear_model import LinearRegression
skl_model = LinearRegression()
skl_model.fit(X_train_scaled, y_train) # Fit on scaled data for fair comparison
y_pred_skl = skl_model.predict(X_test_scaled)

mse_skl = mean_squared_error(y_test, y_pred_skl)
r2_skl = r2_score(y_test, y_pred_skl)

print("\n--- Scikit-learn Linear Regression (for comparison) ---")
print(f"Intercept (beta_0): {skl_model.intercept_:.4f}")
print("Coefficients (beta_i for each feature):")
for feature, coef in zip(X.columns, skl_model.coef_):
    print(f"  {feature}: {coef:.4f}")
print(f"Mean Squared Error (MSE): {mse_skl:.4f}")
print(f"R-squared (R2 Score): {r2_skl:.4f}")